In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ND = 64  # 1辺の差分分割数

In [3]:
# 計算1ステップ
def step(cc):
    res = cc + delt * ( dcdt(cc) + c_flu*(2 * np.random.rand(ND, ND) - 1.) )	#濃度場の時間発展（濃度揺らぎ導入）
    # res = cc + delt * dcdt(cc)
    res = comp(res)
    return res

In [4]:
# 濃度場の時間微分
def dcdt(cc):
    return Mc*laplacian(diff_pot(cc))

In [5]:
# 拡散ポテンシャルの計算
def diff_pot(cc):
    c1=1.0-cc
    c2=cc
    mu_chem=L0*(c1-c2)+np.log(c2)-np.log(c1)		#化学ポテンシャル差
    mu_surf=-2.0*kappa_c*laplacian(cc)		#濃度勾配のポテンシャル
    mu_str=2.0*eta*eta*y100*(c2-c0) 		#弾性ポテンシャル

    Tc=TcFe*c1+TcCr*c2+c1*c2*(TcCrFe0+TcCrFe1*(c2-c1)) #Curie温度
    d2Tc=-TcFe+TcCr+2.0*TcCrFe1*c1*c2+(c1-c2)*(TcCrFe0+TcCrFe1*(c2-c1)) #TcのCr組成微分
    Bc=BcFe*c1+BcCr*c2+BcCrFe0*c1*c2 #１原子当たりの磁化の強さ（ボーア磁子で無次元化）
    d2Bc=-BcFe+BcCr+BcCrFe0*(c1-c2) #BcのCr組成微分


    Tc = np.where(Tc<0.0,-Tc,Tc)
    d2Tc = np.where(Tc<0.0,-d2Tc,d2Tc)
    Bc = np.where(Bc<0.0,-Bc,Bc)
    d2Bc = np.where(Bc<0.0,-d2Bc,d2Bc)

    tau=temp/Tc #τの定義

    ftau = np.where(tau<=1.0,
            1.0-1.0/D_mag*(79.0/140.0/p_mag/tau+474.0/497.0*(1.0/p_mag-1.0)*( np.power(tau,3.0)/6.0+np.power(tau,9.0)/135.0+np.power(tau,15.0)/600.0) ),
            -1.0/D_mag*(np.power(tau,-5.0)/10.0+np.power(tau,-15.0)/315.0+np.power(tau,-25.0)/1500.0)
            )

    dftau = np.where(tau<=1.0,
            -1.0/D_mag*(-79.0/140.0/p_mag/tau/tau+474.0/497.0*(1.0/p_mag-1.0)*( np.power(tau,2.0)/2.0+np.power(tau,8.0)/15.0+np.power(tau,14.0)/40.0) ),
            1.0/D_mag*(np.power(tau,-6.0)/2.0+np.power(tau,-16.0)/21.0+np.power(tau,-26.0)/60.0)
            )

    mu_mag=ftau/(Bc+1.0)*d2Bc-dftau*tau/Tc*d2Tc*np.log(Bc+1.0) #磁気過剰エネルギーのポテンシャル

    return mu_chem + mu_surf + mu_str + mu_mag
    #return mu_chem+mu_surf+mu_str

In [6]:
# 二次元のラプラシアン
def laplacian(array):
    result = -4.0 * array
    result += np.roll(array, 1, 0)
    result += np.roll(array, -1, 0)
    result += np.roll(array, 1, 1)
    result += np.roll(array, -1, 1)
    return result

In [7]:
# 濃度補正
def comp(cc):
    res = cc - (np.average(cc) - c0)  # 濃度場の収支補正
    res = np.clip(res, 0.00001, 0.99999)  # 濃度場の範囲補正
    return res

In [8]:
#初期場の生成
def init():
    c = c0 + 0.01 * (2.0 * np.random.rand(ND, ND) - 1.0)
    return c


al = 30.0 * 1.e-9
b1 = al / ND

time1 = 0
time1max = 20000

delt = 0.02

temp = 673.0
rr = 8.3145
rtemp = rr * temp

c0 = 0.5

cmob = 1.0

L0_0=20500.0		#相互作用パラメータ(J/mol)
L0_1=-9.68			#L=L0_0+L0_1*T
L0=(L0_0+L0_1*temp)/rtemp 	#RTで規格化

kappa_c = 2.0e-15 / b1 ** 2 / rtemp

a01=0.28664e-9 #格子定数(bcc Fe) (m)
atomNo=2.0		#単位胞内の原子数
vm0=6.02e23*a01*a01*a01/atomNo	#モル体積

eta=0.00614	#格子ミスマッチ

c11a=2.3310e+11 	#bccFeの弾性定数(Pa)
c12a=1.3544e+11
c44a=1.1783e+11

c11b=3.500e+11 	#bccCrの弾性定数(Pa)
c12b=0.678e+11
c44b=1.008e+11

c11a=c11a*vm0/rtemp #RTで無次元化
c12a=c12a*vm0/rtemp
c44a=c44a*vm0/rtemp
c11b=c11b*vm0/rtemp
c12b=c12b*vm0/rtemp
c44b=c44b*vm0/rtemp

c11=(1.0-c0)*c11a+c0*c11b	#合金の弾性率
c12=(1.0-c0)*c12a+c0*c12b
c44=(1.0-c0)*c44a+c0*c44b

y100=c11+c12-2.0*(c12*c12/c11)	#弾性率の関数Y<100>

TcFe=1043.0 #Curie温度関連の係数
TcCr=-311.5
TcCrFe0=1650.0
TcCrFe1=550.0

BcFe=2.22
BcCr=-0.008
BcCrFe0=-0.85 #ボーア磁子関連の係数

p_mag=0.4 #(bcc)
D_mag=518.0/1125.0+11692.0/15975.0*(1.0/p_mag-1.0)

Mc=c0*(1.0-c0) #拡散の易動度
c_flu=0.1 #ポテンシャルにおけるゆらぎ

c = init()

In [ ]:
for time1 in range(time1max):
    c = step(c)  # 拡散方程式の計算

    if time1 % 200 == 0:  # 200タイムステップごとに描画
        plt.clf()  # 描画内容クリア
        plt.imshow(c)  # 濃度場の描画
        plt.clim(0, 1)  # カラースケールの最大、最小の設定
        plt.pause(0.01)  #0.01秒間表示